# <center>GeoSpatial use case: EX1 High-resolution hybrid land-cover mapping</center>

# Purpose

The objetive of the tutorial is to explain the Experiment 1 of the **GeoSpatial use case** in the H2020 CloudButton project. The main goal of the project is to create a Serverless Data Analytics Platform that aims to “democratize big data” by overly simplifying the overall life cycle and programming model thanks to serverless technologies. The idea is to tap into stateless functions to enable radically-simpler, more user-friendly data processing systems. The CloudButton platform will seamlessly   integrate a Serverless Compute Engine, a Mutable Shared Data Middleware, and new Serverless Cloud Programming Abstractions on top.

The first experiment is based in a high-resolution hybrid land-cover mapping through multi-temporal imagery from Sentinel-2. This high resolution raster image is classified with a hybrid land-cover classification based in a supervised classifier, Naive Bayes, and the serverless technology thanks to the framework lithops.

#  Index
The tutorial contains the following sections properly explain.

**1. Setup**

    1.1. Set up docker
    
    1.2. Importing libraries
    
    1.3. Downloading Sentinel-2 images
    
    
**2. Dataset processing**

    2.1. Merge
    
    2.2. Mask
    
    2.3. Tiling
    
    2.4. Selection
    
    2.5. Stackering
    
**3. Machine Learning - Naive Bayes model**

    3.1. Load a raster image
    
    3.2. Obtain the pixels of the training areas
    
    3.3. Build the training for the classifier
    
    3.4. Train the model
    
    3.5. Prediction
    
    3.6. Export and import trained model 
    
**4. Serverless Implementation - Lithops**

    4.1. Configuration lithops
    
    4.2. Map method
    
    4.3. Iterdata and execution

The **first** section will specify all the requeriments and obtain the data from Sentinel-2 that is going to be processed. The **second** part dataset processing, will work with all the satelite images in order to prepare the classification algorithm. A moder is gonna be train and saved for the future use in serverless. In the **third** and last part, all the object from the IBM Cloud are gonna be parallel classified based in the Naive Bayes trained model thanks to the framework lithops.

# 1. Setup

### 1.1. Set up docker

To work with all the libraries in a **jupyter notebook** we will run a **docker** built for the experiment. It is necesarry to download <a href="https://www.docker.com/products/docker-desktop">docker desktop</a>, once it is done, we download and run the image of the docker that its gonna be used.

> docker pull mavsonnen/jupy-notebook:geosv2<br>
> docker run mavsonnen/jupy-notebook:geosv2

A **local directory** can be connected to a docker container creating a volume using <i>-v option</i>, change <i>local_path</i>:

> docker run -p 8888:8888 -v local_path:/home/jovyan/work mavsonnen/jupy-notebook:geosv2

### 1.2. Importing libraries

The following libraries are necessary:

In [1]:
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from shapely.geometry import mapping
from IPython.display import Image
from IPython.core.display import HTML 
from shapely.geometry import mapping
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.windows import Window
from rasterio.plot import reshape_as_raster
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import lithops
import shapely
import shutil
import pickle
import joblib
import gdal
import time
import glob
import ogr
import os
import io

print("Libraries imported succesfully")

Libraries imported succesfully


### 1.3. Downloading Sentinel-2 images

We will work with Sentinel-2 images with Level-2A, which already has an atmospheric correction. The data will be downloaded from **Copernicus Open Access Hub** in the following <a href="https://scihub.copernicus.eu/dhus/#/home">link</a>. 

Copernicus Open Access Hub offers us many searching options like sensing and ingestion period. It is important to work with Sentinel-2 level 2A satelite images, product type has to be set like  <i>S2MSl2A</i>.

Using the right click on the map we create the area of interest. As it is showed in the following image, Copernicus will show  all the raster images that overlap the area of interest allowing the user to view the details and choose which ones to **download**.

Every product from copernicus comes with a large amount of files and information. In every product it will be needed three **Surface Reflectance** (SRE) bands, the ones which end up in the form: "SRE_B2", "SRE_B3" and "SRE_B8".

![Image of Yaktocat](https://scihub.copernicus.eu/twiki/pub/SciHubUserGuide/GraphicalUserInterface/gui-10.jpg)

# 2. Dataset processing

## 2.1. Merge
We proceed to merge all the raster of the same band. Finally we will have as many raster as bands we are working with, in particular three.

**Define directories**

 - <b>input</b>: path to images for the mosaic process
 
 - <b>output</b>: output folder where the mosaic image will be saved


In [2]:
tmp = '/home/jovyan/work/' 

input = f"{tmp}Dataset_valencia/Dataset_sentinel_bands/band3/SENTINEL2*.tif" 
output = f"{tmp}Dataset_valencia/Dataset_by_bands/Mosaic/com_val_band3.tif" 

**Read** all Sentinel2 raster images from input path

In [3]:
files = glob.glob(input)
filesmosaic=[]
files

['/home/jovyan/work/Dataset_valencia/Dataset_sentinel_bands/band3/SENTINEL2A_20200927-110006-031_L2A_T30TYK_C_V2-2_SRE_B3.tif',
 '/home/jovyan/work/Dataset_valencia/Dataset_sentinel_bands/band3/SENTINEL2A_20200927-110009-082_L2A_T30TXK_C_V2-2_SRE_B3.tif',
 '/home/jovyan/work/Dataset_valencia/Dataset_sentinel_bands/band3/SENTINEL2B_20200902-105949-025_L2A_T30TYL_C_V2-2_SRE_B3.tif',
 '/home/jovyan/work/Dataset_valencia/Dataset_sentinel_bands/band3/SENTINEL2B_20200912-110031-737_L2A_T30SYH_C_V2-2_SRE_B3.tif',
 '/home/jovyan/work/Dataset_valencia/Dataset_sentinel_bands/band3/SENTINEL2B_20201012-110036-590_L2A_T30SXH_C_V2-2_SRE_B3.tif',
 '/home/jovyan/work/Dataset_valencia/Dataset_sentinel_bands/band3/SENTINEL2B_20201101-110017-667_L2A_T30SYJ_C_V2-2_SRE_B3.tif',
 '/home/jovyan/work/Dataset_valencia/Dataset_sentinel_bands/band3/SENTINEL2B_20201101-110020-834_L2A_T30SXJ_C_V2-2_SRE_B3.tif']

In [26]:
for i in files:
    src = rasterio.open(i)
    filesmosaic.append(src)

mosaic, out_trans = merge(filesmosaic)

Copy and set **metadata**:

In [5]:
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                          "height": mosaic.shape[1],
                          "width": mosaic.shape[2],
                          "transform": out_trans,
                          "crs": 'epsg:32630'
                          }
                         )

**Save file** in output directory:

In [6]:
with rasterio.open(output, "w", **out_meta) as dest:
    dest.write(mosaic)  

We **repeat** the merge section with **all the bands** we are gonna use for the analyses. In particular we will use three bands B3, B4 and B8.

 ## 2.2 Mask

We mask the raster images of each band with the administrative delimiter of the region of study. We will perform it with Comunidad Valenciana.

**Define directories**

 - <b>layer_path</b>: input layer path for masking
 
 - <b>raser_path</b>: path of the raster image to be masked
  
 - <b>out_tif</b>: output path to save the mask image

In [7]:
layer_path = f'{tmp}Dataset_valencia/Dataset_files/comunidad_valenciana.shp'
raster_path = f'{tmp}Dataset_valencia/Dataset_by_bands/Mosaic/com_val_band3.tif'
out_tif = f'{tmp}Dataset_valencia/Dataset_by_bands/Mask/com_val_band3_mask.tif'

Proccess to make high performance image warping

In [8]:
OutTile = gdal.Warp(out_tif, 
                    raster_path, 
                    cutlineDSName=layer_path,
                    cropToCutline=True,
                    dstNodata = 0)

We **repeat** the mask section with **all the bands** we are gonna use for the analyses.

## 2.3 Tiling

### Create grid

Creating a grid for creating smaller raster images out of the bigger raster image.

**Define directories**

 - <b>path_to_layer</b>: path to shp file of the desire extent
 
 - <b>output</b>: output folder where the grid will be saved
 
 - <b>spacing</b>: spacing between polygons in the grid
  
 - <b>epsg</b>: define epsg 

In [9]:
path_to_layer = f'{tmp}Dataset_valencia/Dataset_files/comunidad_valenciana.shp'  
output = f'{tmp}Dataset_valencia/Dataset_files/grid_com_val.shp'
spacing = 16000
epsg = 25830 

Define **size of the extent** to be sure the bounding box created has all the set point inside and it is multiple of the spacing.

In [10]:
gdf = gpd.read_file(path_to_layer)
xmin, ymin, xmax, ymax = gdf.total_bounds # bounds of the total geometry

ytop = np.ceil(np.ceil(ymax) / spacing) * spacing
ybottom = np.floor(np.floor(ymin) / spacing) * spacing
xright = np.ceil(np.ceil(xmax) / spacing) * spacing
xleft = np.floor(np.floor(xmin) / spacing) * spacing

# Defining number of rows and columns
rows = int((ytop - ybottom) / spacing)
cols = int((xright - xleft) / spacing)

A for loop for **creating all polygons** of the grid

In [11]:
polygons = []
it = 0
listfid = []

for i in np.arange(xleft, xright, spacing):
    xleft = i
    xright = xleft + spacing
    ytop_backup = ytop
    for j in np.arange(ytop, ybottom, -spacing):
        ytop = j
        ybottom = ytop - spacing

        polygon = shapely.geometry.Polygon([
            (xleft, ytop),
            (xright, ytop),
            (xright, ybottom),
            (xleft, ybottom)
        ]
        )
        polygons.append(polygon)
        listfid.append(it)
        it += 1
    ytop = ytop_backup

Set metadata and write it into the disk

In [12]:
srs = f"epsg:{epsg}"
fid = pd.DataFrame({"fid_id": listfid})
grid = gpd.GeoDataFrame(fid, geometry=polygons, crs={"init": srs})

print("Writing grid into disk")
grid.to_file(output, driver="GPKG")

/opt/conda/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Writing grid into disk


### Get layer extent
Return the extent of layer with GPKG or ESRI Shapefile formats

 - <b>layer_path</b>: Path to the layer

In [13]:
def get_layerextent(layer_path):
     # check if it is gpgk or shp file
    longitud = len(layer_path.split("."))
    driver_name = layer_path.split(".")[longitud - 1]
    if driver_name == "gpkg":
        driver = ogr.GetDriverByName("GPKG")
    if driver_name == "shp":
        driver = ogr.GetDriverByName("ESRI Shapefile")
        
     # open and get the extent
    ds = driver.Open(layer_path)
    xmin, xmax, ymin, ymax = ds.GetLayer().GetExtent()
    extent = f"{xmin}, {ymin}, {xmax}, {ymax}"

    del ds
    raster_extent=extent

### Layer within raster
check if a layer is inside the raster

 - <b>raster_extent</b>: extent of the raster
 
 - <b>layer_geom</b>: geometry of the layer
 
 - <b>lesser_lextetn</b>: If True a smaller extent is evaluated

In [14]:
def layer_within_raster(raster_extent, layer_geom, lesser_lextent=False):

    rxmin, rxmax, rymin, rymax = raster_extent
    lxmin, lxmax, lymin, lymax = layer_geom.GetEnvelope()

    if lesser_lextent:
        # Getting a smaller bounding box
        lxmin = lxmin + 100
        lxmax = lxmax - 100
        lymin = lymin + 100
        lymax = lymax - 100

    i = 0
    if lxmin >= rxmin:  # 1. upper left corner
        i += 1
    if lymax <= rymax:  # 2. upper right corner
        i += 1
    if lxmax <= rxmax:  # 3. lower right corner
        i += 1
    if lymin >= rymin:  # 4. lower left corner
        i += 1

    if i == 4:
        out = True
    else:
        out = False
    return (out)

### Naming convention
Creates naming based on the raster name and geometries: date_xmin-ymax_sentineltile_band

 - <b>raster_path</b>: path to raster file
 
 - <b>geometry</b>: geometry of the feature


In [15]:
def naming_convention(raster_path, geometry):

    # xmin, xmax, ymin, ymax
    xmin, xmax, ymin, ymax = geometry.GetEnvelope()
    splitted = raster_path.split("/")
    len_splitted = len(splitted)
    name_tmp1 = splitted[len_splitted - 1]
    name = name_tmp1.split(".")[0]
    name_splitted = name.split("_")
    if len(name_splitted) < 3:
        outaname = f"{name}_{float(xmin)}-{float(ymax)}"
    else:
        sent_tile = name_splitted[0]
        band = name_splitted[2]
        date_tmp = name_splitted[1]
        date = date_tmp.split("T")[0]

        outaname = f"{date}_{float(xmin)}-{float(ymax)}_{sent_tile}_{band}"
    return (outaname)

### Reproject
Function to reproject an image to a desire espg

 - <b>image</b>: path to raster image
 
 - <b>output_folder</b>: output folder where the output image will be saved
 
 - <b>epsg_to</b>: coordinate epsg code to reproject into. 25830 by deafult.


In [16]:
def reproject(image, output_folder, epsg_to=25830):
    
    splitted = image.split("/")
    lenout = len(splitted)
    out_name = splitted[lenout-1]
    output = f"{output_folder}/reprojeted_{out_name}"

    dataset = gdal.Open(image)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(epsg_to)
    vrt_ds = gdal.AutoCreateWarpedVRT(dataset, None, srs.ExportToWkt(), gdal.GRA_NearestNeighbour)

    # cols = vrt_ds.RasterXSize
    # rows = vrt_ds.RasterYSize
    gdal.GetDriverByName("GTiff").CreateCopy(output, vrt_ds)

### Masking tiles
It creates tiles from a raster image based on a grid previously created

 - <b>layer_tiles</b>: path to grid
 
 - <b>raster_path</b>: path to raster
 
 - <b>output_folder</b>: path to output folder
  
 - <b>field</b>: Field with cut tiles with
  
 - <b>naming</b>: Apply naming rule (function define before)
  
 - <b>extent</b>: Cut with extent  
 
 - <b>lesser_lextent</b>: create an smaller extent 
 
 - <b>reproyectar</b>: If True, reprojection is applied
   
 - <b>epsg</b>: EPSG code of the srs to reproject into

In [17]:
def masking_tiles(layer_tiles,
                  raster_path,
                  output_folder,
                  field="fid_id",
                  naming=True,
                  extent=False, #Falta desarrollar extent para .tif
                  lesser_lextent=False,
                  reproyectar=False,
                  epsg=25830
                  ):
    
    if os.path.exists(output_folder) is False:
        os.mkdir(output_folder)

    if reproyectar:
        raster_path2 = raster_path
        raster_path = reproject(raster_path, output_folder, epsg_to=epsg)
        print(raster_path)

    driver=ogr.GetDriverByName("GPKG")
    ds = driver.Open(layer_tiles)
    layer = ds.GetLayer()
    i=0
    
    print('Creating masking tiles files')
    for feature in layer: 
        i=i+1
        geom = feature.geometry()
        fid = feature.GetField(field)
        if naming:
            if reproyectar:
                out_name = naming_convention(raster_path2, geom)
            else:
                out_name = naming_convention(raster_path, geom)
        else:
            out_tmp = raster_path.split("/")
            out_tmp2 = out_tmp[len(out_tmp) - 1]
            out_name = out_tmp2.split(".")[0]

        output = f"{output_folder}/{out_name}.tif"
        print(f'Done successfully: {out_name}')
        
        if extent:
            raster_extent = get_layerextent(raster_path)
            #raster_extent = get_rasterextent(raster_path) ORIGINAL
            sepuede = layer_within_raster(raster_extent, geom, lesser_lextent=lesser_lextent)

            if sepuede:
                xmin, xmax, ymin, ymax = geom.GetEnvelope()
                lextent = [xmin, ymin, xmax, ymax]

                ds2 = gdal.Warp(output,
                                raster_path,
                                format="GTiff",
                                outputBounds=lextent)

                del ds2

        else:
            ds2 = gdal.Warp(output,
                            raster_path,
                            format="GTiff",
                            cutlineDSName=layer_tiles,
                            cutlineWhere=f"{field} = '{fid}'",
                            cropToCutline=True)
            del ds2

    layer.ResetReading()
    ds.FlushCache()

    del ds

Define path to files and execute

In [18]:
layer_tiles = f'{tmp}Dataset_valencia/Dataset_files/grid_com_val.gpkg'
raster_path = f'{tmp}Dataset_valencia/Dataset_by_bands/Mask/com_val_band3_mask.tif'
output_folder = f'{tmp}Dataset_valencia/Dataset_by_bands/Tiling_b3'

masking_tiles(layer_tiles, raster_path, output_folder)


Creating masking tiles files
Done successfully: val_624000.0-4528000.0_com_band3
Done successfully: val_624000.0-4512000.0_com_band3
Done successfully: val_624000.0-4496000.0_com_band3
Done successfully: val_624000.0-4480000.0_com_band3
Done successfully: val_624000.0-4464000.0_com_band3
Done successfully: val_624000.0-4448000.0_com_band3
Done successfully: val_624000.0-4432000.0_com_band3
Done successfully: val_624000.0-4416000.0_com_band3
Done successfully: val_624000.0-4400000.0_com_band3
Done successfully: val_624000.0-4384000.0_com_band3
Done successfully: val_624000.0-4368000.0_com_band3
Done successfully: val_624000.0-4352000.0_com_band3
Done successfully: val_624000.0-4336000.0_com_band3
Done successfully: val_624000.0-4320000.0_com_band3
Done successfully: val_624000.0-4304000.0_com_band3
Done successfully: val_624000.0-4288000.0_com_band3
Done successfully: val_624000.0-4272000.0_com_band3
Done successfully: val_624000.0-4256000.0_com_band3
Done successfully: val_624000.0-424

Done successfully: val_736000.0-4480000.0_com_band3
Done successfully: val_736000.0-4464000.0_com_band3
Done successfully: val_736000.0-4448000.0_com_band3
Done successfully: val_736000.0-4432000.0_com_band3
Done successfully: val_736000.0-4416000.0_com_band3
Done successfully: val_736000.0-4400000.0_com_band3
Done successfully: val_736000.0-4384000.0_com_band3
Done successfully: val_736000.0-4368000.0_com_band3
Done successfully: val_736000.0-4352000.0_com_band3
Done successfully: val_736000.0-4336000.0_com_band3
Done successfully: val_736000.0-4320000.0_com_band3
Done successfully: val_736000.0-4304000.0_com_band3
Done successfully: val_736000.0-4288000.0_com_band3
Done successfully: val_736000.0-4272000.0_com_band3
Done successfully: val_736000.0-4256000.0_com_band3
Done successfully: val_736000.0-4240000.0_com_band3
Done successfully: val_736000.0-4224000.0_com_band3
Done successfully: val_736000.0-4208000.0_com_band3
Done successfully: val_736000.0-4192000.0_com_band3
Done success

## 2.4 Selection

**Define directories**

 - <b>input_path</b>: path where all the tiling raster images are located
 
 - <b>output_path</b>: path for the selection of the raster images


In [19]:
input_path = f'{tmp}Dataset_valencia/Dataset_by_bands/Tiling_b8/'
output_path = f'{tmp}Dataset_valencia/Dataset_by_bands/Tiling_b8_selection/'

criteria = 'valenciana*.tif'
all_files = glob.glob(input_path+criteria)

Loop checking which raster images are merely water and discarding them.

In [20]:
for raster_path in all_files:
    raster = rasterio.open(raster_path)
    data_matrix = raster.read()

    file_name = raster_path.split('/')[len(raster_path.split('/'))-1]

    if np.unique(data_matrix).any()!=0:
        print(f'Processing the file: {file_name}')
        tmp = rasterio.open(output_path + file_name, 'w', driver='Gtiff',
                                 width=raster.width, height=raster.height,
                                 count=raster.count,
                                 crs=raster.crs,
                                 transform=raster.transform,
                                 dtype=raster.dtypes[0]
                                 )
        tmp.write(raster.read(1),1)
        
        if raster.count != 1:
            tmp.write(raster.read(2),2)
            tmp.write(raster.read(3),3)
            
        tmp.close()

Processing the file: valenciana_624000.0-4368000.0_comunidad_band8.tif
Processing the file: valenciana_624000.0-4384000.0_comunidad_band8.tif
Processing the file: valenciana_624000.0-4400000.0_comunidad_band8.tif
Processing the file: valenciana_624000.0-4432000.0_comunidad_band8.tif
Processing the file: valenciana_624000.0-4448000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4320000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4336000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4352000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4368000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4384000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4400000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4416000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4432000.0_comunidad_band8.tif
Processing the file: valenciana_640000.0-4448000.0_comunidad_band8.tif
Proces

Processing the file: valenciana_752000.0-4464000.0_comunidad_band8.tif
Processing the file: valenciana_752000.0-4480000.0_comunidad_band8.tif
Processing the file: valenciana_752000.0-4496000.0_comunidad_band8.tif
Processing the file: valenciana_752000.0-4512000.0_comunidad_band8.tif
Processing the file: valenciana_752000.0-4528000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4288000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4304000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4320000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4448000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4464000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4480000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4496000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4512000.0_comunidad_band8.tif
Processing the file: valenciana_768000.0-4528000.0_comunidad_band8.tif
Proces

## 2.5. Stackering

The aim of this section is to **stack all the selected bands** in a unique raster image for the future classification.

Define que path where the previous tiling raster images are store.

In [21]:
files_b3 = f'{tmp}Dataset_valencia/Dataset_by_bands/Tiling_b3_selection'
files_b4 = f'{tmp}Dataset_valencia/Dataset_by_bands/Tiling_b4_selection'
files_b8 = f'{tmp}Dataset_valencia/Dataset_by_bands/Tiling_b8_selection'

Based on the criteria we select all the bands for the stackering process and get all the coordinates for the name convention.

In [22]:
criteria = '/valenciana*.tif'
all_name_file=[]
all_files=[]

for name_band in [files_b3,files_b4,files_b8]:
    input_directory = name_band + criteria
    list_files = glob.glob(input_directory)
    all_files.append(list_files)

for x in list_files:   
    name_file = x.split("/")[len(x.split("/"))-1].split("_")[1]
    all_name_file.append(name_file)

Based on the **name convetion** we create all the folder to store the bands in each one.

In [23]:
for name in all_name_file:
    try:
        os.mkdir(f"{tmp}/Dataset_valencia/Dataset_by_bands/Stackering/{name}")
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

We **move bands** to each folder that was created.

In [24]:
for i in range(0,len(all_files[0])):
    shutil.move(all_files[0][i],f"{tmp}/Dataset_valencia/Dataset_by_bands/Stackering/{all_name_file[i]}")
    shutil.move(all_files[1][i],f"{tmp}/Dataset_valencia/Dataset_by_bands/Stackering/{all_name_file[i]}")
    shutil.move(all_files[2][i],f"{tmp}/Dataset_valencia/Dataset_by_bands/Stackering/{all_name_file[i]}")

We take the **metadata information** from one band and update it for later define it over the stacked raster image.

In [25]:
list = os.listdir(f"{tmp}/Dataset_valencia/Dataset_by_bands/Stackering/")

for i in range(0,len(list)):
    coord_bands = glob.glob(f"{tmp}/Dataset_valencia/Dataset_by_bands/Stackering/{list[i]}/val*")
    #print(coord_bands)
    
    raster_meta = rasterio.open(coord_bands[0])
    meta = raster_meta.meta 
    meta.update(count = len(coord_bands))

    img_stack = f"{tmp}/Dataset_valencia/Dataset_by_bands/Stackering_output/{list[i]}.tif"
    
    with rasterio.open(img_stack, 'w', **meta) as dst:
        for id, layer in enumerate(coord_bands, start=1):
            with rasterio.open(layer) as src1:
                dst.write_band(id, src1.read(1))
    print(f'Stackering process finished, raster image {list[i]} created.')
    

FileNotFoundError: [Errno 2] No such file or directory: "<closed DatasetWriter name='/home/jovyan/work/Dataset_valencia/Dataset_by_bands/Tiling_b8_selection/valenciana_784000.0-4512000.0_comunidad_band8.tif' mode='w'>/Dataset_valencia/Dataset_by_bands/Stackering/"

We **verify** the raster image has been properly built after the stackering process.

In [ ]:
full_data = rasterio.open(img_stack)
print('The stakering image has the shape: {} and it is composed of {} bands.'.format(full_data.shape,full_data.count))
show(full_data)

# 3. Machine Learning - Naive Bayes model

## 3.1. Obtain the pixels of the training areas

The raster is gonna be splitted in different areas for the future training of the classifier. For this step we need to **create an ERSI Shapefile** with different training polygons, the ones that are gonna be used for the training process. We can help from QGIS or SNAP software for creation of those areas.

![](https://community.hexagongeospatial.com/t5/image/serverpage/image-id/4240iF1A4D500227ABC21/image-size/original?v=1.0&px=-1)

Define the **directory** to the training areas shapefile.

In [ ]:
name = 'valenciana_704000.0-4400000.0_comunidad_mask'
train_areas = gpd.read_file(f'{tmp}training/areas_{name}.shp')
print('The training area has the shape: {} with a coordinate reference system {}.'.format(train_areas.shape,train_areas.crs))

<u>Note</u>: if it does not have the same coordinate system of the dataset use to_crs and set the correct epsg. Now we generate a list of geometries and check a random feature to verify.

In [ ]:
geoms = train_areas.geometry.values
geometry = geoms[0]
geoms[10]

The **raster values are extracted** within the training areas

In [ ]:
img_stack = tmp + 'tiling/' + name +'.tif'
full_data = rasterio.open(img_stack)

In [ ]:
feature = [mapping(geometry)]
out_img_extr, out_transform = mask(full_data, feature, crop=True)
out_transform

### 2.3. Build the training for the classifier

Based on the previous data we have been analysing, we create two arrays with the **feature and labels data** for the training of the classifier. First we define two empty arrays:

In [ ]:
X = np.array([], dtype=np.int8).reshape(0,3) # pixels
y = np.array([], dtype=np.string_) # labels

**Extract the value** of our raster within the training polygons. We get <i>X</i> and <i>y</i>, the features and layers arrays respectively.

In [ ]:
with rasterio.open(img_stack) as src:
    band_count = src.count
    
    for index, geom in enumerate(geoms):      
        feature = [mapping(geom)] # transform to gjson format for an easier manipulation.      
        out_img_extr, out_transform = mask(src, feature, crop=True) # mask return pixel in to array    
        out_img_extr_trimmed = out_img_extr[:,~np.all(out_img_extr == 0, axis=0)] # remove all pixels with 0 values 
        out_img_extr_trimmed = out_img_extr_trimmed[:,~np.all(out_img_extr_trimmed == 255, axis=0)] # remove all pixels with 255 values  
        out_img_extr_reshaped = out_img_extr_trimmed.reshape(-1, band_count) # reshape the array
        
        y = np.append(y,[train_areas["layer"][index]] * out_img_extr_reshaped.shape[0]) # add the labels to array
        X = np.vstack((X,out_img_extr_reshaped)) # stack the pixels to array

print('We define X as the array that stores the features with a shape {} and y as the labels array with a shape {}.'.format(X.shape,y.shape))        

**Check the different classes** that are define in our raster image.

In [ ]:
print('The raster image contain {} classes which are the following: \n {}'.format(np.unique(train_areas["layer"]).size, np.unique(train_areas["layer"])))

### 2.4.Train the model


Once all the data have been proccess we are ready to **train the classifier**, in this section the classifier will learn from our data in order to make an accurate classification for the next raster image to be classified.

In [ ]:
gnb = GaussianNB()
gnb.fit(X, y)
print("Naive Bayes trainned.")

In order to achieve a better classification the model have to be **train with more raster images**. The dataset used to train the model have to be diverse and paying attention not to fall in overfiting or underfitting.

### 2.5. Prediction 

We proceed to make the prediction, however it is gonna be done with serverless technology in the following sections so migth skip this point 2.5.

We advance to the **prediction of the raster**. Based on what the classifier has learn it will make a classification with a maximun of 10 different classes, the ones are shown bellow.

- artificial surface
- water surface
- transition vegetation
- conifers
- hardwood forest
- natural grasslands
- mediterranean scrub
- dry farming land
- agricultural areas
- wetlands
- burned areas

It is defined a function <i>str_class_to_int</i> that assigns the classes to indices and some modification of the raster shape to be able to perform the prediction.

In [ ]:
def str_class_to_int(class_array):
    class_array[class_array == 'superficie_artificial'] = 1
    class_array[class_array == 'bosque_de_coniferas'] = 2
    class_array[class_array == 'bosque_de_frondosas'] = 3
    class_array[class_array == 'zonas_agricolas'] = 4
    class_array[class_array == 'vegetacion_esclerofila'] = 5
    class_array[class_array == 'pastizales_naturales'] = 6
    class_array[class_array == 'matorral_boscoso_de_transicion'] = 7
    class_array[class_array == 'zonas_quedamas'] = 8
    class_array[class_array == 'zonas_humedas'] = 9
    class_array[class_array == 'superficie_agua'] = 10
    return(class_array.astype(int))

If the computing cost is making the process significantly slower, if it is necessary to work with a part of the image, in that case don't read the whole image defining the **position**: col_off, row_off, width, height. For <u>example</u>: <i>src.read()[:, 1500: 2000, 1500 : 2500]</i>.

In [ ]:
with rasterio.open(img_stack) as src:
    img = src.read()[:, : , :]

**Reshape** the complete image and in a long 2d matrix to be able to visualize it

In [ ]:
reshaped_img = reshape_as_image(img)

For each pixel of our we **perform a prediction** and reshape our classified image in a 2d matrix to be able to see it

In [ ]:
prediction = gnb.predict(reshaped_img.reshape(-1, 3))
prediction = prediction.reshape(reshaped_img[:, :, 0].shape)

From the shapefile initially convert the **strings to numpy matrix**.

In [ ]:
prediction = str_class_to_int(prediction)
print(np.unique(prediction))
print(prediction.shape)

**Show prediction** to have an overview of the raster classified.

In [ ]:
show(prediction, cmap='terrain')

We can **save** the array as a raster image .tif which is the classified image. First we change the format to <i>int32</i> and then create an output raster image from the prediction.

In [ ]:
y = prediction.astype(np.int32) 

SalidaR = rasterio.open(tmp + 'out3.tif','w',
                   driver='Gtiff',
                   width = 1599,
                   height = 1599,
                   count=1,
                   crs = full_data.crs,  
                   transform = full_data.transform, 
                   dtype='int32')
SalidaR.write(y,1)
SalidaR.close()

print("Raster image succesfully created!")

### 2.6. Export and import  trained model 

Save the **Machine Learning model** based in Naive Bayes through <i>joblib</i> and import it. By doing so we will be able to upload it to IBM_COS. 


In [ ]:
model = gnb
name_model = 'naive_bayes_model.sav' 
joblib.dump(model, tmp + '/COS/' + name_model)

print("Trained model saved")

In [ ]:
loaded_model = joblib.load(tmp + '/COS/' + name_model)

print("Train model imported")

# 3. Serverless Implementation - Lithops

We have reached the serverless implementation thanks to the framework **lithops**. The model which was trained before will be used to classify different raster images in order to extract the different classes.

All files that are gonna be used for the classification, raster images and models, require to be uploaded in **IBM Cloud Object Storage**. The raster images are need to be storage in the same bucket for the parallel classification.

![Image of Yaktocat](https://github.com/lithops-cloud/lithops/blob/master/docs/images/lithops_flat_cloud_1.png?raw=true)

### 3.1 Configuration of lithops

Working with lithops require a configuration file like the one express below. More info can be found in the official **<a href="https://github.com/lithops-cloud/lithops">github</a> of Lithops**. 

 <i>
    config = { 'lithops' : {'storage_bucket' : 'BUCKET_NAME'},
    
           'ibm_cf':  {'endpoint': 'ENDPOINT',
                      'namespace': 'NAMESPACE',
                      'api_key': 'API_KEY'},
    
           'ibm_cos': {'region': 'REGION',
                      'api_key': 'API_KEY'}}

The **config** file with all the parameters is uploaded below

In [ ]:
import json
config_path = open("/home/jovyan/work/config.txt")
config_read = config_path.read()
config = json.loads(config_read)

### 3.2 Map method

Use **map_classification** to run multiple function executors and proceed to the classification in parallel. The output is the raster image of the classification.

In [ ]:
def map_classification(obj, ibm_cos):
    
    with rasterio.open(obj.data_stream) as src:
        img = src.read() #[:, 150: 200, 150 : 250] - col_off, row_off, width, height
        reshaped_img = reshape_as_image(img)
        shapefile = ibm_cos.get_object(Bucket='geospatial-first-experiment', Key='naive_bayes_model.sav')['Body']
        loaded_model = joblib.load(io.BytesIO(shapefile.read())) 
        prediction = loaded_model.predict(reshaped_img.reshape(-1, 3))  # for each pixel of our we perform a prediction
        prediction = prediction.reshape(reshaped_img[:, :, 0].shape)

        prediction[prediction == 'superficie_artificial'] = 1
        prediction[prediction == 'bosque_de_coniferas'] = 2
        prediction[prediction == 'bosque_de_frondosas'] = 3
        prediction[prediction == 'zonas_agricolas'] = 4
        prediction[prediction == 'vegetacion_esclerofila'] = 5
        prediction[prediction == 'pastizales_naturales'] = 6
        prediction[prediction == 'matorral_boscoso_de_transicion'] = 7
        prediction[prediction == 'zonas_quedamas'] = 8
        prediction[prediction == 'zonas_humedas'] = 9
        prediction[prediction == 'superficie_agua'] = 10
        
        prediction= (prediction.astype(int))
        y = prediction.astype(np.int32)
        
    return  prediction

### 3.3 Iterdata and execution

As mentioned, all the raster images to be parallel clasified must be **stored** in the **same bucket**.

In [ ]:
data_location = 'cos://large-scale-process'

An instance of the **executor** is created, afterwars the function previously developed is executed by the **map_classification** and finally storing the **output** in the variable <i>results</i>. 

<u>Note:</u> For Python 3.7 use mavsonnen/lithops-py37:latest + For Python 3.7 use mavsonnen/jdsampe:nogdal

In [ ]:
seg_start = time.time()

fexec = lithops.FunctionExecutor(config = config, runtime = 'mavsonnen/jdsampe:nogdal', runtime_memory = 2048)
fexec.map(map_classification, data_location)
results = fexec.get_result()

print(f'Parallel classification thanks to lithops finished successfully in {time.time() - seg_start} s')

All the raster image parallel classified are **shown** above:

In [ ]:
for img in results:
    show(img, cmap='terrain')